In [2]:
import os
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from datetime import datetime as dt
from datetime import timedelta
import time
import pandas as pd
import psutil

from oceanoobsbrasil.db import GetData
from oceanoobsbrasil.utils import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from oceanoobsbrasil.tides.ebn import EbnTide

In [8]:
bot = EbnTide()

In [9]:
bot.get()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="menu_623"]"}
  (Session info: headless chrome=93.0.4577.63)


In [11]:
from oceanoobsbrasil.tides.hydro_met_curua import HydroMetCurua

ModuleNotFoundError: No module named 'oceanoobsbrasil.tides.hydro_met_curua'

In [30]:
from oceanoobsbrasil.tides.get_tide_ilhafiscal import HydroMetIlhaFiscal

In [33]:
bot = HydroMetIlhaFiscal()

In [34]:
bot.get()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="login"]"}
  (Session info: headless chrome=93.0.4577.63)


In [22]:
bot.url